---
title: IDs from STEREO
order: 11
---

See following notebooks for details:

- [Magnetic field data](./mag.ipynb)
- [State data](./state.ipynb)

STEREO magnetic field is already in RTN coordinates, so no need to transform it.

Download data using `pyspedas`, but load it using `pycdfpp` (using `pyspedas` to load the data directly into `xarray` is very slow)

Note STEREO-A plastic data file is "Non compliant ISTP file: No data variable found, this is suspicious", `-9.999999848243207e+30` instead of `-1e+31` is used as fill value.

```
wget -r -np -nH -R "index.html*" https://spdf.gsfc.nasa.gov/pub/data/stereo/ahead/l2/plastic/1dmax/1min/201{1,2,3,4,5,6}/
```

In [1]:
import speasy as spz
from space_analysis.utils.speasy import Variables
from space_analysis.io.cdf import cdf2pl
from discontinuitypy.datasets import IDsDataset
from beforerr.polars import pl_norm
from discontinuitypy.utils.basic import resample

from datetime import timedelta
import polars as pl
import polars.selectors as cs

from sunpy.time import TimeRange
import pyspedas

from tqdm import tqdm

25-Feb-24 01:01:20: UserWarning: Traceback (most recent call last):
  File "/Users/zijin/micromamba/envs/psp_conjunction/lib/python3.11/site-packages/pdpipe/__init__.py", line 85, in <module>
    from . import skintegrate
  File "/Users/zijin/micromamba/envs/psp_conjunction/lib/python3.11/site-packages/pdpipe/skintegrate.py", line 20, in <module>
    from sklearn.base import BaseEstimator
ModuleNotFoundError: No module named 'sklearn'


25-Feb-24 01:01:20: UserWarning: pdpipe: Scikit-learn or skutil import failed. Scikit-learn-dependent pipeline stages will not be loaded.

25-Feb-24 01:01:20: UserWarning: Traceback (most recent call last):
  File "/Users/zijin/micromamba/envs/psp_conjunction/lib/python3.11/site-packages/pdpipe/__init__.py", line 105, in <module>
    from . import nltk_stages
  File "/Users/zijin/micromamba/envs/psp_conjunction/lib/python3.11/site-packages/pdpipe/nltk_stages.py", line 19, in <module>
    import nltk
ModuleNotFoundError: No module named 'nltk'


25-Feb-2

In [2]:
import speasy as spz
entries = spz.core.cache.entries()
# drop internal entries
entries = list(filter(lambda e: '__internal__' not in e, entries))
print(entries[::-1])

['cda/WI_PLSP_3DP/MOM.P.MAGT3/2019-04-12T00:00:00+00:00', 'cda/WI_PLSP_3DP/MOM.P.MAGT3/2019-04-11T12:00:00+00:00', 'cda/WI_PLSP_3DP/MOM.P.MAGT3/2019-04-11T00:00:00+00:00', 'cda/WI_PLSP_3DP/MOM.P.MAGT3/2019-04-10T12:00:00+00:00', 'cda/WI_PLSP_3DP/MOM.P.MAGT3/2019-04-10T00:00:00+00:00', 'cda/WI_PLSP_3DP/MOM.P.MAGT3/2019-04-09T12:00:00+00:00', 'cda/WI_PLSP_3DP/MOM.P.MAGT3/2019-04-09T00:00:00+00:00', 'cda/WI_PLSP_3DP/MOM.P.MAGT3/2019-04-08T12:00:00+00:00', 'https://cdaweb.gsfc.nasa.gov/tmp/wsX7LELp/wi_plsps_3dp_20190408120000_20190412115954.cdf', 'https://cdaweb.gsfc.nasa.gov/tmp/wss6lQ5G/wi_plsps_3dp_20190408120000_20190412115954.cdf', 'https://cdaweb.gsfc.nasa.gov/tmp/wsgD6oUy/wi_plsps_3dp_20190408120000_20190412115954.cdf', 'https://cdaweb.gsfc.nasa.gov/tmp/ws2WbWGk/wi_plsps_3dp_20190408120000_20190412115954.cdf', 'https://cdaweb.gsfc.nasa.gov/tmp/wsw6URGt/wi_plsps_3dp_20190408120000_20190412115954.cdf', 'https://cdaweb.gsfc.nasa.gov/tmp/wsrsvJ7P/wi_plsps_3dp_20190408120000_201904121159

In [3]:
spz.update_inventories()

In [4]:
timerange = ["2011-08-25", "2016-06-30"]
# timerange = ["2011-08-25", "2011-09-01"]
mission = "STA"
ts = timedelta(seconds=1)
tau = timedelta(seconds=60)

provider = 'archive/local'
mag_dataset = "STA_L1_MAG_RTN"
mag_parameters = ["BFIELD"]

# plasma_dataset = "STA_L2_MAGPLASMA_1M"
# plasma_parameters= ["Np", "Cone_Angle", "Vt_Over_V_RTN", "Vp_RTN", "Tp"]

plasma_dataset = 'STA_L2_PLA_1DMAX_1MIN'
density_col = 'proton_number_density'
vec_cols = ['proton_Vr_RTN', 'proton_Vt_RTN', 'proton_Vn_RTN']
temperature_col = 'proton_temperature'
plasma_parameters = [density_col] + vec_cols + [temperature_col]

fmt = 'arrow'
fname = f"../../../data/05_reporting/events.{mission}.ts_{ts.total_seconds():.2f}s_tau_60s.{fmt}"

## Downloading files

In [ ]:
def download(timerange):
    
    files = pyspedas.stereo.plastic(timerange, downloadonly=True)

    for file in files:
        parquet_file = file.replace('.cdf', f'.{fmt}')
        cdf2pl(file, plasma_parameters).collect().write_ipc(parquet_file)
        
    mag_files = pyspedas.stereo.mag(timerange, downloadonly=True)
    
    return files
    
        
# download(timerange)

## Loading data

In [7]:
mag_vars = Variables(
    provider = provider,
    dataset=mag_dataset,
    parameters=mag_parameters,
    timerange=timerange,
).retrieve_data()

KeyboardInterrupt: 

In [ ]:
def get_and_process_data(timerange):
    mag_vars = Variables(
        provider = provider,
        dataset=mag_dataset,
        parameters=mag_parameters,
        timerange=timerange,
    ).retrieve_data()

    bcols = mag_vars.data[0].columns[:3]

    mag_data = mag_vars.to_polars().drop('BTotal')
    
    files = pyspedas.stereo.plastic(timerange, downloadonly=True, no_update=True)
    parquet_files = [file.replace('.cdf', f'.{fmt}') for file in files]

    plasma_data = pl.scan_ipc(parquet_files).with_columns(
        plasma_density = pl.when(pl.col(density_col) <0 ).then(None).otherwise(pl.col(density_col)),
        plasma_speed=pl_norm(vec_cols), ).rename({temperature_col:'plasma_temperature'}
    )
    
    return IDsDataset(
        mag_data=mag_data.pipe(resample, every=ts),
        plasma_data=plasma_data,
        tau=tau,
        ts=ts,
        bcols=bcols,
        vec_cols=vec_cols,
        density_col="plasma_density",
        speed_col="plasma_speed",
        temperature_col='plasma_temperature',
    ).find_events(return_best_fit=False).update_candidates_with_plasma_data()

In [ ]:
ids : list[pl.DataFrame] = []
for _tr in tqdm(TimeRange(timerange).split(8)):
    
    _timerange = [_tr.start.value, _tr.end.value]
    _id = get_and_process_data(_timerange)
    ids.append(_id.events)
    
id = pl.concat(ids, how='vertical_relaxed').select(cs.datetime(), cs.duration(), cs.numeric())
id.write_ipc(fname)